# Importy

In [38]:

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload

### DataLoader
import DataLoader.xAPIConnector
reload(DataLoader.xAPIConnector)
from DataLoader.xAPIConnector import *

import DataLoader.DataLoader
reload(DataLoader.DataLoader)
from DataLoader.DataLoader import *

from DataLoader.config import user_id, pwd


### Data
import Data.SymbolParser
reload(Data.SymbolParser)
from Data.SymbolParser import parse_symbols


### TrendAnalysis
import TrendAnalysis.MACD
reload(TrendAnalysis.MACD)
from TrendAnalysis.MACD import MACD

import TrendAnalysis.MA
reload(TrendAnalysis.MA)
from TrendAnalysis.MA import MA

import TrendAnalysis.RSI
reload(TrendAnalysis.RSI)
from TrendAnalysis.RSI import RSI


### TechFunctions
from TechFunctions.Technicalities import position_encoding

# Właściwy kod

In [34]:
Symbols = parse_symbols()
for key, val in Symbols.items():
    print(f"{key}: {val}")

gold: ['NEM.US', 'AU.US', 'RGLD.US', 'CDE.US', 'SA.US', 'CMCL.US', 'IDR.US', 'CTGO.US', 'FTCO.US', 'USAU.US']
copper: ['SCCO.US', 'FCX.US', 'MTAL.US', 'IE.US']
aluminum: ['AA.US', 'CENX.US', 'CSTM.US', 'KALU.US']
coal: ['HCC.US', 'AMR.US', 'ARCH.US', 'SXC.US', 'METC.US']
silver: ['AG.US', 'MAG.US', 'AYASF.US', 'SVM.US']


In [35]:
symbols = Symbols['coal']
start, interval = '2020-01-01 00:00:00', '1D'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval, verbose=False)
data

[BŁĄD] Nie pobrano SXC.US
[BŁĄD] Nie pobrano METC.US


,HCC.US,AMR.US,ARCH.US
Date,,,
2020-01-02,NaN,8.82,73.35
2020-01-03,NaN,8.63,73.42
2020-01-06,NaN,8.78,72.24
2020-01-07,NaN,8.38,71.81
2020-01-08,NaN,7.55,70.59
...,...,...,...
2025-01-23,55.75,197.59,NaN
2025-01-24,54.61,194.49,NaN
2025-01-27,53.22,189.37,NaN


In [36]:
for s in [-1.0, 0.0, 1.0]:
    s_str = position_encoding[int(s)]
    
    model = MA(data)
    print(f"Sygnały {s_str} z modelu MA:\n{(model.Decision()==s).sum()}")

    model = MACD(data)
    print(f"Sygnały {s_str} z modelu MACD:\n{(model.Decision()==s).sum()}")

    model = RSI(data)
    print(f"Sygnały {s_str} z modelu RSI:\n{(model.Decision()==s).sum()}")
    
    print()

Sygnały sell z modelu MA:
HCC.US     29
AMR.US     28
ARCH.US    28
dtype: int64
Sygnały sell z modelu MACD:
HCC.US     42
AMR.US     44
ARCH.US    53
dtype: int64
Sygnały sell z modelu RSI:
HCC.US     18
AMR.US     22
ARCH.US    25
dtype: int64

Sygnały hold z modelu MA:
HCC.US     1217
AMR.US     1220
ARCH.US    1220
dtype: int64
Sygnały hold z modelu MACD:
HCC.US     1191
AMR.US     1187
ARCH.US    1170
dtype: int64
Sygnały hold z modelu RSI:
HCC.US     1216
AMR.US     1202
ARCH.US    1193
dtype: int64

Sygnały buy z modelu MA:
HCC.US     30
AMR.US     28
ARCH.US    28
dtype: int64
Sygnały buy z modelu MACD:
HCC.US     43
AMR.US     45
ARCH.US    53
dtype: int64
Sygnały buy z modelu RSI:
HCC.US      9
AMR.US     14
ARCH.US    17
dtype: int64



In [37]:
model.Decision()

,HCC.US,AMR.US,ARCH.US
Date,,,
2020-01-02,0.0,0.0,0.0
2020-01-03,0.0,0.0,0.0
2020-01-06,0.0,0.0,0.0
2020-01-07,0.0,0.0,0.0
2020-01-08,0.0,0.0,0.0
...,...,...,...
2025-01-23,0.0,0.0,0.0
2025-01-24,0.0,0.0,0.0
2025-01-27,0.0,0.0,0.0
